In [1]:
%load_ext autoreload
%autoreload 1

%aimport basics
%aimport nn_config
%aimport enc_dec


from basics import *
from nn_config import *
from enc_dec import *
%matplotlib inline

callhome es-en word level configuration
translating es to en
vocab size, en=51, fr=51


In [2]:
xp = cuda.cupy if gpuid >= 0 else np

In [3]:
text_data = pickle.load(open(text_data_dict, "rb"))

In [4]:
model = SpeechEncoderDecoder(SPEECH_DIM, vocab_size_en, num_layers_enc, num_layers_dec,
                               hidden_units, gpuid, attn=use_attn)
if gpuid >= 0:
    cuda.get_device(gpuid).use()
    model.to_gpu()

In [5]:
log_train_fil_name, text_fname, dev_fname, test_fname

('es_speech_to_en_char_model/train_17394sen_4-2layers_128units_es_en_speech2text_batches_callhome_es_en_1.log',
 {'en': '../../corpora/callhome/uttr_fa_vad_wavs/train.en',
  'fr': '../../corpora/callhome/uttr_fa_vad_wavs/speech_train.es'},
 {'en': '../../corpora/callhome/uttr_fa_vad_wavs/dev.en',
  'fr': '../../corpora/callhome/uttr_fa_vad_wavs/speech_dev.es'},
 {'en': '../../corpora/callhome/uttr_fa_vad_wavs/test.en',
  'fr': '../../corpora/callhome/uttr_fa_vad_wavs/speech_test.es'})

In [6]:
%aimport nmt_trials
from nmt_trials import *

Starting experiment
num sentences=4 and num epochs=10


In [7]:
buckets, bucket_lengths = populate_buckets()

  1%|▏         | 168/13137 [00:00<00:07, 1677.83it/s]

Splitting data into 35 buckets, each of width=24


100%|██████████| 13137/13137 [00:06<00:00, 2053.25it/s]

Saving bucket data


In [9]:
25*32, 24*35

(800, 840)

In [17]:
batch_training(BATCH_SIZE, BATCH_SIZE, buckets, bucket_lengths, SPEECH_BUCKET_WIDTH, 0)

  0%|          | 0/4 [00:00<?, ?it/s]

(4, 24, 120) (4, 24, 120) (4, 12)
(24, 4, 120) (24, 4, 120) (12, 4)
trying encode
L0_enc before (24, 4, 120)
12 (24, 4, 120) 2
lat (4, 128)
out (4, 128)
lat concat (4, 256)
lat (4, 128)
out (4, 128)
lat concat (4, 256)
(1, 4, 256) (4, 256)
lat (4, 128)
out (4, 128)
lat concat (4, 256)
(2, 4, 256) (4, 256)
lat (4, 128)
out (4, 128)
lat concat (4, 256)
(3, 4, 256) (4, 256)
lat (4, 128)
out (4, 128)
lat concat (4, 256)
(4, 4, 256) (4, 256)
lat (4, 128)
out (4, 128)
lat concat (4, 256)
(5, 4, 256) (4, 256)
lat (4, 128)
out (4, 128)
lat concat (4, 256)
(6, 4, 256) (4, 256)
lat (4, 128)
out (4, 128)
lat concat (4, 256)
(7, 4, 256) (4, 256)
lat (4, 128)
out (4, 128)
lat concat (4, 256)
(8, 4, 256) (4, 256)
lat (4, 128)
out (4, 128)
lat concat (4, 256)
(9, 4, 256) (4, 256)
lat (4, 128)
out (4, 128)
lat concat (4, 256)
(10, 4, 256) (4, 256)
lat (4, 128)
out (4, 128)
lat concat (4, 256)
(11, 4, 256) (4, 256)
L0_enc out (12, 4, 256)
L1_enc before (12, 4, 256)
6 (12, 4, 256) 2
lat (4, 128)
out (4,

AssertionError: 

In [ ]:
# predict(s=0, num=1, cat="train", display=True, plot=False, p_filt=0, r_filt=0)

In [ ]:
# print(b" ".join(get_ids(text_data["train"]["041.004"]["en"])[0]))

In [ ]:
# forward_states = model[model.lstm_enc[-1]].h
# backward_states = model[model.lstm_rev_enc[-1]].h

In [ ]:
# model.enc_states = F.concat((forward_states, backward_states), axis=1)

In [ ]:
# train_loop(num_training=1000, num_epochs=1)

In [ ]:
wavs = []
wavs.append(xp.load(os.path.join(speech_dir, "041.001"+speech_extn)))
wavs.append(xp.load(os.path.join(speech_dir, "041.004"+speech_extn)))

In [ ]:
[l.shape for l in wavs]

In [ ]:
zero_arr = xp.zeros((3,120), dtype=xp.float32)

In [ ]:
wavs[1].dtype, zero_arr.dtype

In [ ]:
# xp.vstack((wavs[1], zero_arr))
wavs[1].shape, zero_arr.shape

In [ ]:
wavs.append(pad_array(wavs[1], 1000, at_start=True))

In [ ]:
xp.concatenate((wavs[1], zero_arr), axis=0).shape

In [ ]:
wavs[1][:3].shape

In [ ]:
xp.pad(wavs[1], pad_width=(0,100), mode='constant', constant_values=0)

In [ ]:
def display_buckets(bucket_lengths, width_b = SPEECH_BUCKET_WIDTH):
    headings = ("ix", "len b", "num", "max fr", "avg fr", "max en", "avg en")
    print("{0:3s} | {1:5s} | {2:5s} | {3:6s} | {4:8s} | {5:6s} | {6:8s}".format(*headings))
    print("\n".join(["{0:3d} | {1:5d} | {2:5d} | {3:6d} | {4:8.0f} | {5:6d} | {6:8.0f}".format(i[0], (i[0]+1)*width_b, *i[1]) for i in list(bucket_lengths.items())]))


def populate_buckets(width_b = SPEECH_BUCKET_WIDTH, 
                     num_b = SPEECH_NUM_BUCKETS, 
                     speech=True, 
                     num_sent=NUM_TRAINING_SENTENCES,
                     filname_b=speech_bucket_data_fname,
                     cat="train", display=False):
    
    buckets = [[] for i in range(num_b)]
    
    print("Splitting data into {0:d} buckets, each of width={1:d}".format(num_b, width_b))

    with tqdm(total=num_sent) as pbar:
        for i, sp_fil in enumerate(sorted(list(text_data[cat].keys()))[:num_sent]):

            fr_ids, en_ids, speech_feat = get_data_item(sp_fil, cat=cat)

            len_en = len(en_ids)
            len_fr = len(fr_ids)
            len_speech = len(speech_feat)
            
            indx_b = min(num_b-1, len_speech // width_b)

            buckets[indx_b].append((sp_fil, len_speech, len_fr, len_en))

            pbar.update(1)
    
    bucket_lengths = {i:(len(l), 
                    max(l, key=lambda t:t[2])[2],
                    np.mean([i[2]for i in l]),
                    max(l, key=lambda t:t[3])[3],
                    np.mean([i[3]for i in l]))
                     for i, l in enumerate(buckets)}

    if display:
        display_buckets(bucket_lengths)

    # Saving bucket data
    print("Saving bucket data")
    pickle.dump(buckets, open(filname_b, "wb"))
    return buckets, bucket_lengths

In [ ]:
b, bucket_lengths = populate_buckets(display=True)

In [ ]:
b[1][:5]

In [ ]:
haha = [0 for i in range(10)]

In [ ]:
d = haha[:30]
print(len(d))

In [ ]:
bucket_lengths